Needs a few dependencies, at minimum do:

pip install lxml geopandas pycountry fsspec

In [10]:
import pandas as pd
import pycountry

In [11]:

# Load the Wikipedia page
url = "https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table"
tables = pd.read_html(url)

# Assuming the third table is the one with the all-time medal data
medal_table = tables[2]  # Adjust this index if necessary

print(medal_table)

                 Team (IOC code)  No. Summer  No. Winter  No. Games
0           American Samoa (ASA)          10           2         12
1                  Andorra (AND)          13          13         26
2                   Angola (ANG)          11           0         11
3      Antigua and Barbuda (ANT)          12           0         12
4                    Aruba (ARU)          10           0         10
..                           ...         ...         ...        ...
66       North Borneo (NBO)[NBO]           1           0          1
67  Republic of China (ROC)[ROC]           3           0          3
68               Saar (SAA)[SAA]           1           0          1
69        North Yemen (YAR)[YAR]           2           0          2
70        South Yemen (YMD)[YMD]           1           0          1

[71 rows x 4 columns]


def get_country_name(noc_code):
    try:
        return pycountry.countries.get(alpha_3=noc_code).name
    except AttributeError:
        return None

countries_with_no_medals_names = [get_country_name(code) for code in countries_with_no_medals]

import geopandas as gpd
import matplotlib.pyplot as plt
import fsspec

# Load the world shapefile
url = "https://www.naturalearthdata.com/http//www.naturalearthdata.com/" \
      "download/110m/cultural/ne_110m_admin_0_countries.zip"
with fsspec.open(f"simplecache::{url}") as file:
    gdf = gpd.read_file(file)


# Filter countries with no medals
no_medals_df = world[world['name'].isin(medal_table)]

# Plot the map
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
world.plot(ax=ax, color='lightgrey')
no_medals_df.plot(ax=ax, color='red')

plt.title('Countries with No Olympic Medals')
plt.show()